In [18]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_lgb
importlib.reload(preprocesamiento)
importlib.reload(model_lgb)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

# Experimento 7: 
- LGBM
- Estandarizacion del target
- Usando funcion entrenamiento: semillerio_en_prediccion
- Mismas variables
- Pesos: (log o max?)
- sqlite:///optuna_studies_v16.db
- Kaggle =  


##### Levantamos el dataset con target ya calculado

In [2]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target_transformado_201912.csv", sep=',', encoding='utf-8')
print("Dataset sin transformar tenia esto: (31362, 19)")
df.shape

Dataset sin transformar tenia esto: (31362, 19)


(31362, 35)

In [3]:
columnas_baseline = df.columns.tolist()
columnas_baseline

['product_id',
 'periodo',
 'nacimiento_producto',
 'muerte_producto',
 'mes_n',
 'total_meses',
 'producto_nuevo',
 'ciclo_de_vida_inicial',
 'cat1',
 'cat2',
 'cat3',
 'brand',
 'sku_size',
 'stock_final',
 'tn',
 'plan_precios_cuidados',
 'cust_request_qty',
 'cust_request_tn',
 'target',
 'tn_mean',
 'tn_std',
 'tn_zscore',
 'stock_final_mean',
 'stock_final_std',
 'stock_final_zscore',
 'cust_request_qty_mean',
 'cust_request_qty_std',
 'cust_request_qty_zscore',
 'cust_request_tn_mean',
 'cust_request_tn_std',
 'cust_request_tn_zscore',
 'tn_log',
 'stock_final_log',
 'cust_request_qty_log',
 'cust_request_tn_log']

##### Preprocesamiento a la minima expresión :)

In [4]:
# ##### aplicamos OHE
df = preprocesamiento.aplicarOHE(df)
df.shape

(31362, 178)

### Feature Engineering

##### Neural Prophet

In [5]:
neural_prophet_fe = pd.read_csv("./datasets/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 181)

##### Prophet

In [6]:
prophet_fe = pd.read_csv("./datasets/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 187)

##### FE Moviles

In [7]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
df = feature_engineering.get_rolling_medians(df, "tn", 201912)
df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df.shape

(31362, 749)

In [8]:
df = feature_engineering.get_lags(df, "cust_request_qty", 201912)
df = feature_engineering.get_delta_lags(df, "cust_request_qty", 24)
df = feature_engineering.get_rolling_means(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_stds(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_mins(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_maxs(df, "cust_request_qty", 201912)
df.shape

(31362, 1204)

In [9]:
df = feature_engineering.get_lags(df, "stock_final", 201912)
df = feature_engineering.get_delta_lags(df, "stock_final", 24)
df = feature_engineering.get_rolling_means(df, "stock_final", 201912)
df = feature_engineering.get_rolling_stds(df, "stock_final", 201912)
df = feature_engineering.get_rolling_mins(df, "stock_final", 201912)
df = feature_engineering.get_rolling_maxs(df, "stock_final", 201912)
df.shape

(31362, 1659)

Features Diana

In [10]:
df = feature_engineering.calcular_diferencia_con_medias_moviles(df)
df = feature_engineering.calcular_ratios_con_medias_moviles(df)
df.shape

##### FE Moviles sobre otras variables

In [56]:
# #  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

#  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty')

##### FE Calendario

In [11]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 1720)

##### FE sobre FE

In [12]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 1749)

##### Variables Exogenas

In [13]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 1752)

##### Nuevas FE

In [15]:
df = feature_engineering.create_ratio_features(df)
df = feature_engineering.enhance_lifecycle_features(df)
# df = feature_engineering.create_category_features(df)
df = feature_engineering.create_regime_features(df)
df = feature_engineering.create_nonlinear_trends(df)
df = feature_engineering.create_temporal_interactions(df)
df = feature_engineering.create_asymmetric_window_features(df)
df = feature_engineering.recomendaciones_deepseek(df)
df = feature_engineering.get_nuevas_features(df)
df.shape

(31362, 1789)

##### Elimino aquellas que no sirven

In [ ]:
# importantes = pd.read_csv("./feature_importance/exp04_3.csv", sep=',', encoding='utf-8')
# no_importantes = importantes[importantes['importance'] <= 100]
# no_importantes = no_importantes[~no_importantes['feature'].isin(columnas_baseline)]
# no_importantes

,feature,importance
67,tn_vs_prev_year,100
68,tn_delta_lag5_lag8,99
69,tn_rolling_min_4,96
70,tn_lag_5,95
71,tn_rolling_std_6,95
...,...,...
677,dia_del_year,0
678,cat2_TE,0
680,cat2_PIEL1,0
681,cat2_OTROS,0


In [ ]:
# cols_a_eliminar = no_importantes.feature.unique()
# print(f"Antes de eliminar: {df.shape[1]} columnas")
# df = df.drop(columns=cols_a_eliminar, errors='ignore')
# print(f"Después de eliminar: {df.shape[1]} columnas")

Antes de eliminar: 1144 columnas
Después de eliminar: 684 columnas


Eliminar object/categorical columnas

In [16]:
df = df.select_dtypes(exclude=['datetime', 'datetime64', 'object'])

Train Test Split

In [17]:
train = df[df['periodo'] <= 201912]
test = df[df['periodo'] == 201912]

Entrenamiento

In [19]:
model_lgb.optimizar_con_optuna_con_semillerio_db(train, version="v19", n_trials=500)


Para visualizar los resultados en tiempo real:
1. Abre otra terminal y ejecuta:
   optuna-dashboard sqlite:///optuna_studies_v19.db
2. Abre en tu navegador: http://127.0.0.1:8080/


[I 2025-07-05 18:25:05,506] A new study created in RDB with name: lightgbm_optimization_v19
[I 2025-07-05 18:32:32,527] Trial 0 finished with value: 0.5334971393323207 and parameters: {'num_leaves': 47, 'learning_rate': 0.2536999076681772, 'feature_fraction': 0.892797576724562, 'bagging_fraction': 0.8795975452591109, 'bagging_freq': 2, 'lambda_l1': 2.5348407664333426e-07, 'lambda_l2': 3.3323645788192616e-08, 'min_child_samples': 45, 'max_depth': 7, 'max_bin': 383, 'min_data_in_leaf': 21, 'extra_trees': True, 'early_stopping_rounds': 18, 'path_smooth': 0.18182496720710062, 'min_gain_to_split': 0.09170225492671691}. Best is trial 0 with value: 0.5334971393323207.


Mejor trial hasta ahora: RMSE=0.533497, Parámetros={'num_leaves': 47, 'learning_rate': 0.2536999076681772, 'feature_fraction': 0.892797576724562, 'bagging_fraction': 0.8795975452591109, 'bagging_freq': 2, 'lambda_l1': 2.5348407664333426e-07, 'lambda_l2': 3.3323645788192616e-08, 'min_child_samples': 45, 'max_depth': 7, 'max_bin': 383, 'min_data_in_leaf': 21, 'extra_trees': True, 'early_stopping_rounds': 18, 'path_smooth': 0.18182496720710062, 'min_gain_to_split': 0.09170225492671691}


[I 2025-07-05 18:41:18,584] Trial 1 finished with value: 1.0468776170540237 and parameters: {'num_leaves': 41, 'learning_rate': 0.05958389350068958, 'feature_fraction': 0.7727780074568463, 'bagging_fraction': 0.7873687420594125, 'bagging_freq': 7, 'lambda_l1': 1.8007140198129195e-07, 'lambda_l2': 4.258943089524393e-06, 'min_child_samples': 25, 'max_depth': 6, 'max_bin': 414, 'min_data_in_leaf': 36, 'extra_trees': False, 'early_stopping_rounds': 11, 'path_smooth': 0.6075448519014384, 'min_gain_to_split': 0.08526206184364576}. Best is trial 0 with value: 0.5334971393323207.


Mejor trial hasta ahora: RMSE=0.533497, Parámetros={'num_leaves': 47, 'learning_rate': 0.2536999076681772, 'feature_fraction': 0.892797576724562, 'bagging_fraction': 0.8795975452591109, 'bagging_freq': 2, 'lambda_l1': 2.5348407664333426e-07, 'lambda_l2': 3.3323645788192616e-08, 'min_child_samples': 45, 'max_depth': 7, 'max_bin': 383, 'min_data_in_leaf': 21, 'extra_trees': True, 'early_stopping_rounds': 18, 'path_smooth': 0.18182496720710062, 'min_gain_to_split': 0.09170225492671691}


...

[I 2025-07-06 18:19:06,985] Trial 129 finished with value: 0.5415751282395478 and parameters: {'num_leaves': 68, 'learning_rate': 0.2785469609325003, 'feature_fraction': 0.9354563521019831, 'bagging_fraction': 0.8831480269491641, 'bagging_freq': 6, 'lambda_l1': 4.0031789231836083e-07, 'lambda_l2': 0.0027198892172274397, 'min_child_samples': 43, 'max_depth': 7, 'max_bin': 263, 'min_data_in_leaf': 27, 'extra_trees': False, 'early_stopping_rounds': 13, 'path_smooth': 0.7885793654920624, 'min_gain_to_split': 0.15985019316767696}. Best is trial 122 with value: 0.18531639606907524.


Mejor trial hasta ahora: RMSE=0.185316, Parámetros={'num_leaves': 64, 'learning_rate': 0.28065227608035226, 'feature_fraction': 0.9333600565094247, 'bagging_fraction': 0.8598520114855284, 'bagging_freq': 1, 'lambda_l1': 3.0537157049570355e-07, 'lambda_l2': 0.0007720811892991811, 'min_child_samples': 43, 'max_depth': 8, 'max_bin': 281, 'min_data_in_leaf': 27, 'extra_trees': False, 'early_stopping_rounds': 35, 'path_smooth': 0.8373589929930142, 'min_gain_to_split': 0.13429383452015403}


[I 2025-07-06 18:26:20,384] Trial 130 finished with value: 0.4025874270084392 and parameters: {'num_leaves': 72, 'learning_rate': 0.22207845842750307, 'feature_fraction': 0.6300926148936741, 'bagging_fraction': 0.8584653470046534, 'bagging_freq': 1, 'lambda_l1': 1.7172376619691648e-07, 'lambda_l2': 0.0005344853660507952, 'min_child_samples': 46, 'max_depth': 8, 'max_bin': 280, 'min_data_in_leaf': 31, 'extra_trees': False, 'early_stopping_rounds': 36, 'path_smooth': 0.6788386634945727, 'min_gain_to_split': 0.1084605953940655}. Best is trial 122 with value: 0.18531639606907524.


Mejor trial hasta ahora: RMSE=0.185316, Parámetros={'num_leaves': 64, 'learning_rate': 0.28065227608035226, 'feature_fraction': 0.9333600565094247, 'bagging_fraction': 0.8598520114855284, 'bagging_freq': 1, 'lambda_l1': 3.0537157049570355e-07, 'lambda_l2': 0.0007720811892991811, 'min_child_samples': 43, 'max_depth': 8, 'max_bin': 281, 'min_data_in_leaf': 27, 'extra_trees': False, 'early_stopping_rounds': 35, 'path_smooth': 0.8373589929930142, 'min_gain_to_split': 0.13429383452015403}
Estudio guardado en: sqlite:///optuna_studies_v19.db

Mejores hiperparámetros encontrados:
num_leaves: 64
learning_rate: 0.28065227608035226
feature_fraction: 0.9333600565094247
bagging_fraction: 0.8598520114855284
bagging_freq: 1
lambda_l1: 3.0537157049570355e-07
lambda_l2: 0.0007720811892991811
min_child_samples: 43
max_depth: 8
max_bin: 281
min_data_in_leaf: 27
extra_trees: False
early_stopping_rounds: 35
path_smooth: 0.8373589929930142
min_gain_to_split: 0.13429383452015403


(<optuna.study.study.Study at 0x29a8c113550>,
 {'num_leaves': 64,
  'learning_rate': 0.28065227608035226,
  'feature_fraction': 0.9333600565094247,
  'bagging_fraction': 0.8598520114855284,
  'bagging_freq': 1,
  'lambda_l1': 3.0537157049570355e-07,
  'lambda_l2': 0.0007720811892991811,
  'min_child_samples': 43,
  'max_depth': 8,
  'max_bin': 281,
  'min_data_in_leaf': 27,
  'extra_trees': False,
  'early_stopping_rounds': 35,
  'path_smooth': 0.8373589929930142,
  'min_gain_to_split': 0.13429383452015403,
  'objective': 'regression',
  'metric': 'rmse',
  'boosting_type': 'gbdt',
  'verbosity': -1})

Prediccion

In [20]:
df_future = model_lgb.semillerio_en_prediccion_con_pesos(train, test, version="v19")

In [21]:
df_future

,periodo,product_id,target,pred
30476,201912,20001,0.0,1.257930
30477,201912,20002,0.0,0.341550
30478,201912,20003,0.0,0.026131
30479,201912,20004,0.0,0.419695
30480,201912,20005,0.0,0.205230
...,...,...,...,...
31357,201912,21265,0.0,0.389642
31358,201912,21266,0.0,0.190140
31359,201912,21267,0.0,-0.013840
31360,201912,21271,0.0,0.024966


Filtramos los 180 productos

In [22]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df_future = df_future[df_future['periodo'] == 201912]
df_future = df_future[df_future['product_id'].isin(productos_ok['product_id'].unique())]


In [23]:
df_future

,periodo,product_id,target,pred
30476,201912,20001,0.0,1.257930
30477,201912,20002,0.0,0.341550
30478,201912,20003,0.0,0.026131
30479,201912,20004,0.0,0.419695
30480,201912,20005,0.0,0.205230
...,...,...,...,...
31355,201912,21263,0.0,-0.069597
31357,201912,21265,0.0,0.389642
31358,201912,21266,0.0,0.190140
31359,201912,21267,0.0,-0.013840


In [24]:
df_future_copy = df_future.copy()

In [25]:
import os
ruta_archivo = f'./datasets/tn_stats_201912.csv'
    
df_stats = pd.DataFrame()

if os.path.exists(ruta_archivo) and ruta_archivo.endswith('.csv'):
    df_stats = pd.read_csv(ruta_archivo, sep=',')

df_stats

,product_id,tn_mean,tn_std
0,20001,1398.344322,293.975388
1,20002,1009.368178,299.585187
2,20003,889.004243,287.951952
3,20004,671.615383,221.310769
4,20005,644.200514,215.220300
...,...,...,...
1159,21271,0.024268,0.019484
1160,21273,0.057242,0.124272
1161,21274,0.067028,0.096980
1162,21276,0.045447,0.041380


In [26]:
df_future_copy = df_future_copy.merge(df_stats, on=['product_id'], how='left')
df_future_copy

,periodo,product_id,target,pred,tn_mean,tn_std
0,201912,20001,0.0,1.257930,1398.344322,293.975388
1,201912,20002,0.0,0.341550,1009.368178,299.585187
2,201912,20003,0.0,0.026131,889.004243,287.951952
3,201912,20004,0.0,0.419695,671.615383,221.310769
4,201912,20005,0.0,0.205230,644.200514,215.220300
...,...,...,...,...,...,...
775,201912,21263,0.0,-0.069597,0.089233,0.148180
776,201912,21265,0.0,0.389642,0.089541,0.103219
777,201912,21266,0.0,0.190140,0.094659,0.100530
778,201912,21267,0.0,-0.013840,0.092835,0.075836


In [27]:

df_future_copy['tn'] = df_future_copy['pred'] * df_future_copy['tn_std'] + df_future_copy['tn_mean']
df_future_copy

,periodo,product_id,target,pred,tn_mean,tn_std,tn
0,201912,20001,0.0,1.257930,1398.344322,293.975388,1768.144759
1,201912,20002,0.0,0.341550,1009.368178,299.585187,1111.691361
2,201912,20003,0.0,0.026131,889.004243,287.951952,896.528729
3,201912,20004,0.0,0.419695,671.615383,221.310769,764.498492
4,201912,20005,0.0,0.205230,644.200514,215.220300,688.370187
...,...,...,...,...,...,...,...
775,201912,21263,0.0,-0.069597,0.089233,0.148180,0.078920
776,201912,21265,0.0,0.389642,0.089541,0.103219,0.129760
777,201912,21266,0.0,0.190140,0.094659,0.100530,0.113774
778,201912,21267,0.0,-0.013840,0.092835,0.075836,0.091785


Vemos cuantos negativos hay

In [29]:
df_future_copy[df_future_copy['tn'] < 0]

,periodo,product_id,target,pred,tn_mean,tn_std,tn


Reemplazamos los negativos por el promedio de ultimos 12 meses

In [ ]:
# promedio780 = model_lgb.promedio_12_meses_780p()
# df_future = df_future.merge(promedio780, on='product_id', how='left')
# df_future.drop(columns=['target','periodo'], inplace=True)
# df_future.loc[df_future['pred'] < 0, 'pred'] = df_future['tn']
# df_future



,product_id,pred,tn
0,20001,1397.305481,1454.732720
1,20002,1086.538942,1175.437142
2,20003,747.163659,784.976407
3,20004,565.799872,627.215328
4,20005,638.965713,668.270104
...,...,...,...
775,21263,0.029993,0.029993
776,21265,0.791975,0.089541
777,21266,0.094659,0.094659
778,21267,0.092835,0.092835


Guardamos el archivo

In [30]:
# df_future_copy.drop(columns=['tn'], inplace=True)
# df_future_copy.rename(columns={'pred': 'tn'}, inplace=True)
df_future_copy[['product_id', 'tn']].to_csv("./outputs/predicciones_exp_07_lgb_v6.csv", index=False, sep=',')

Ensemble

In [31]:
df_lgb = df_future_copy[['product_id', 'tn']].rename(columns={'tn': 'tn_lgb'})
df_rl = pd.read_csv("./outputs/predicciones_regresion_lineal_v1.csv", sep=',', encoding='utf-8').rename(columns={'tn': 'tn_rl'})
df_ensemble = df_lgb.merge(df_rl, on='product_id', how='left')
df_ensemble['tn'] = (df_ensemble['tn_lgb'] + df_ensemble['tn_rl']) / 2
df_ensemble[['product_id', 'tn']].to_csv("./outputs/predicciones_exp_07_lgb_v6_ensemble.csv", index=False, sep=',')

In [32]:
df_lgb = df_future_copy[['product_id', 'tn']].rename(columns={'tn': 'tn_lgb'})
df_rl = pd.read_csv("./outputs/predicciones_regresion_lineal_v1.csv", sep=',', encoding='utf-8').rename(columns={'tn': 'tn_rl'})
df_ag = pd.read_csv("./outputs/prediccion_autogluon_2ventanas.csv", sep=',', encoding='utf-8').rename(columns={'tn': 'tn_ag'})
df_ensemble = df_lgb.merge(df_rl, on='product_id', how='left')
df_ensemble = df_ensemble.merge(df_ag, on='product_id', how='left')
df_ensemble['tn'] = (df_ensemble['tn_lgb'] + df_ensemble['tn_rl'] + df_ensemble['tn_ag']) / 3
df_ensemble[['product_id', 'tn']].to_csv("./outputs/predicciones_exp_07_lgb_v6_ensemble_3models.csv", index=False, sep=',')